In [125]:
import sys
sys.path.append('..')

import constants.file_handler_constants as fh
from constants.user_constants import *
from constants.attraction_constants import *

import os
import glob
import time
import pandas as pd
import numpy as np
import json
import requests
import ast
import math
from sklearn.neighbors import NearestNeighbors

In [126]:
# get a user data (if user api cannot be used)
user_tag_score_vector = np.array(MOCK_USER[1])

'''
example user id of following persona:
- user_persona_nature --> 677e41b2519209b0aaff4d76, 677e41b6519209b0aaff4d7b
- user_persona_temple_historical --> 677e425a519209b0aaff4dda, 677e425d519209b0aaff4ddf
- user_persona_adventure --> 677e4327519209b0aaff4e3e, 677e432a519209b0aaff4e43
- user_persona_art --> 677e544b519209b0aaff4fd6, 677e544e519209b0aaff4fdb
'''

user_id = '677e4327519209b0aaff4e3e'

try:
    API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/user/getUser/{user_id}"
    headers = {
        "Authorization": f"{os.environ['AUTH_BRANCH_API_KEY']}",
        "Content-Type": "application/json"
    }

    res_all_users = requests.get(url=API_ENDPOINT, headers=headers).json()
    user_tag_score_response = res_all_users['attractionTagScore']['attractionTagFields']
    user_tag_score_vector = np.array(list(user_tag_score_response.values()))

except Exception as e:
    print("GET api failed at user id --> ", user_id)
    print(e)

# get attractions data
attraction_tag_score_data = []
attraction_ref = []
try:
    API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/attraction/getAllData"
    res_all_attractions = requests.post(url=API_ENDPOINT).json()
    
    for cur_attraction in res_all_attractions['attractions']:
        cur_tag_score = cur_attraction['attractionTag']['attractionTagFields']
        cur_ref = (cur_attraction['_id'], cur_attraction['name'])

        attraction_tag_score_data.append(list(cur_tag_score.values()))
        attraction_ref.append(cur_ref)
        
except Exception as e:
    print("get all attraction data failed !")
    print(e)

for cur_tag, cur_ref in zip(attraction_tag_score_data, attraction_ref):
    print(cur_ref)
    print(cur_tag)



('675c202f9f570448ebeb4827', 'เลิฟ ไดวิ่ง ภูเก็ต')
[1, 0.8, 0, 0, 0, 0.2, 0.9, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0.2, 0, 0]
('675c202f9f570448ebeb4829', 'Andaman Sea Kayaks')
[0.9, 0.8, 0.1, 0, 0.1, 0.7, 0.9, 0, 0, 0.2, 0.2, 0, 0, 0, 0, 0, 0.1, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0, 0.7, 0.5, 0.2, 0.1]
('675c202f9f570448ebeb482b', 'The Junk Liveaboards')
[0.9, 0.8, 0.1, 0.1, 0.2, 0.7, 0.8, 0.2, 0.1, 0.5, 0.2, 0.2, 0.1, 0.1, 0.1, 0.1, 0.3, 0.9, 0.3, 0.2, 0.8, 0.1, 0.1, 0.1, 0.1, 0.1, 0, 0, 0.2, 0.1, 0.2, 0.2, 0.5, 0.4, 0.2, 0.2]
('675c202f9f570448ebeb482d', 'ACDC Diving')
[0.9, 0.9, 0, 0, 0, 0.2, 0.8, 0, 0, 0.1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2, 0.1, 0, 0]
('675c202f9f570448ebeb482f', 'พาราไดส์ ไดวิ่ง เอเซีย')
[0.9, 0.8, 0, 0, 0, 0.7, 0.8, 0, 0, 0.9, 0.1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0.8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.7, 0.5, 0, 0]
('675c202f9f570448ebeb4831', 'ซันไรซ์ ไดเวอส์')
[0.9, 0

In [127]:
# train KNN Model
attraction_tag_score_matrix = np.array(attraction_tag_score_data.copy())
knn = NearestNeighbors(n_neighbors=30, metric='cosine') # euclidean, cosine
knn.fit(attraction_tag_score_matrix)

NearestNeighbors(metric='cosine', n_neighbors=30)

In [128]:
# find top recommendations
user_tag_score_vector = user_tag_score_vector.reshape(1, -1)
distances, indices = knn.kneighbors(user_tag_score_vector)

print(distances)
print(indices)

# Print the recommendations before normalize to  [0, 0.05, 0.1, 0.15, 0.2, 0.25, ...., 0.95, 1]
res_rec_by_norm = [] 
print("Top Recommendations:")
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. {attraction_ref[idx][1]} (Similarity: {1 - distances[0][i]:.2f})")


[[0.20624524 0.21967123 0.2326978  0.26208484 0.2623672  0.26260617
  0.27448766 0.27735401 0.28071249 0.28548347 0.29222299 0.29619265
  0.30018104 0.30583104 0.3074552  0.30849402 0.31391768 0.31740744
  0.3189537  0.32015205 0.32253352 0.32529719 0.32545613 0.32583797
  0.32721484 0.33050197 0.33442129 0.33984652 0.34209741 0.3440419 ]]
[[137 329 351 131 293 264 267 156 226 143 151 350 182 154  60 225   2 119
  134 125 147 338 314 237 126 160 323 166 190 158]]
Top Recommendations:
1. ภูเก็ตซีแลนด์ (Similarity: 0.79)
2. CrossFit Siam Phuket (Similarity: 0.78)
3. ซุปเปอร์ ไดเวอส์ (Similarity: 0.77)
4. ซี บี ไดฟ์วิ่ง (Similarity: 0.74)
5. Ziplinemax (Similarity: 0.74)
6. ไวล์ด เวค (Similarity: 0.74)
7. Yoga by Niti (Similarity: 0.73)
8. Dive Link Phuket (Similarity: 0.72)
9. The G Beach Club (Similarity: 0.72)
10. PADI Diving Center Diversaint (Similarity: 0.71)
11. Jolly Roger Adventures (Similarity: 0.71)
12. Hanuman World (Similarity: 0.70)
13. KBA Kiteboarding Asia (Similarity: 0.7

In [129]:
distances

array([[0.20624524, 0.21967123, 0.2326978 , 0.26208484, 0.2623672 ,
        0.26260617, 0.27448766, 0.27735401, 0.28071249, 0.28548347,
        0.29222299, 0.29619265, 0.30018104, 0.30583104, 0.3074552 ,
        0.30849402, 0.31391768, 0.31740744, 0.3189537 , 0.32015205,
        0.32253352, 0.32529719, 0.32545613, 0.32583797, 0.32721484,
        0.33050197, 0.33442129, 0.33984652, 0.34209741, 0.3440419 ]])

In [130]:
indices

array([[137, 329, 351, 131, 293, 264, 267, 156, 226, 143, 151, 350, 182,
        154,  60, 225,   2, 119, 134, 125, 147, 338, 314, 237, 126, 160,
        323, 166, 190, 158]])